# Boston, MA Crime Reports Database

In this project we will setup a PostgreSQL database from scratch and fill it with crime data from a csv file.

We will create two user groups:
* readonly: Users in this group only have permission to read data.
* readwrite: Users in this group have permission to read and modify data but not to delete tables.

## Creating Database and Schema

Let's start by creating a database called crimes_db and a schema called crimes where we will store our tables.

In [1]:
import psycopg2
from sqlalchemy import create_engine
conn = psycopg2.connect(dbname = 'postgres', user = 'postgres', password = '123456789')
cur = conn.cursor

In [2]:
#this cell only clears everything so that we can re-exectue the whole notebook
# conn.autocommit = True

# try:
#     cur.execute('''
#         DROP DATABASE IF EXISTS crime_db;
#     ''')

#     cur.execute('''
#         DROP SCHEMA IF EXISTS crimes CASCADE;
#     ''')

#     cur.execute('''
#         DROP TYPE weekday;
#     ''')
    
#     cur.execute('''
#         DROP ROLE readonly;
#     ''')
    
#     cur.execute('''
#         DROP ROLE readwrite;
#     ''')
    
#     cur.execute('''
#         DROP USER data_analyst;
#     ''')
    
#     cur.execute('''
#         DROP USER data_scientist;
#     ''')
        
# except:
#     pass

In [3]:
conn = psycopg2.connect(dbname = 'postgres', user = 'postgres', password = '123456789')
cur = conn.cursor()

conn.autocommit = True #setting autocommit to true is necessary to create a database

cur.execute('''
    CREATE DATABASE crime_db;
''')

conn.close()

Now that the crime_db database exists we can connect to it

In [4]:
conn = psycopg2.connect(dbname = 'postgres', user = 'postgres', password = '123456789')
cur = conn.cursor()

conn.autocommit = True #setting autocommit to true is also necessary to create a schema 

cur.execute('''
    CREATE SCHEMA crimes;
''')

## Exploring the data
Now that we have our database and schema, let's explore our data so that we can easily select the right datatypes to use in our table. We can do so by seeing if the values of a column are numeric or text values and by the amount of unique values it has.

In [5]:
import pandas as pd
df = pd.read_csv('boston.csv')
print(df.shape)
df.head(2)

(298329, 7)


,incident_number,offense_code,description,date,day_of_the_week,lat,long
0,1,619,LARCENY ALL OTHERS,2018-09-02,Sunday,42.357791,-71.139371
1,2,1402,VANDALISM,2018-08-21,Tuesday,42.306821,-71.060300


In [6]:
for col in df.columns:
    print(col, ': ', len(df[col].unique()))

incident_number :  298329
offense_code :  219
description :  239
date :  1177
day_of_the_week :  7
lat :  18177
long :  18177


We should find the maximum length for the description column.

In [7]:
df['description'].map(len).max()

58

## Creating the table

We will create an enumerated datatype named weekday for the day_of_the_week column because there are only 7 possible values.

The primary key will be the incident_number

We'll use INTEGER datatype for the incident_number and the offense_code columns.

Because the description has a maximum of 58 characters we'll use a VARCHAR (100) datatype. This allows slightly larger values that could be eventually added without wasting too much memory.

The date will be represented by the DATE datatype. 

Lat and long columns will be DECIMAL datatypes

In [8]:
cur.execute('''
    CREATE TYPE weekday as ENUM('Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday', 'Saturday', 'Sunday');
''')

cur.execute('''
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week weekday, 
        lat DECIMAL,
        long DECIMAL
    );
''')

## Loading the data

Now that the table is created we can proceed to load the data by using the copy_expert method.

In [9]:
with open('boston.csv') as f:
    cur.copy_expert('''
        COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;''', f)    

We can make sure that the table was filled by properly by loading it into a new pandas dataframe  making sure the shape is the same.

In [10]:
test_df = pd.read_sql('''
    SELECT *
    FROM crimes.boston_crimes;
''', conn)

print(test_df.shape)
test_df.head()

(298329, 7)


,incident_number,offense_code,description,date,day_of_the_week,lat,long
0,1,619,LARCENY ALL OTHERS,2018-09-02,Sunday,42.357791,-71.139371
1,2,1402,VANDALISM,2018-08-21,Tuesday,42.306821,-71.060300
2,3,3410,TOWED MOTOR VEHICLE,2018-09-03,Monday,42.346589,-71.072429
3,4,3114,INVESTIGATE PROPERTY,2018-09-03,Monday,42.334182,-71.078664
4,5,3114,INVESTIGATE PROPERTY,2018-09-03,Monday,42.275365,-71.090361


We can see that both dataframes have the same amount of rows (298329) and columns (7)

## Now let's revoke all public privileges

We revoke all privileges of the public public group on the public schema to ensure that users will not inherit privileges on that schema such as the ability to create tables in the public schema.

We also need to revoke all privileges in the newly created schema. Doing this we make sure privileges are not granted by default.

In [11]:
cur.execute('''
    REVOKE ALL ON SCHEMA public FROM public;
''')

cur.execute('''
    REVOKE ALL ON DATABASE crime_db FROM public;
''')

## Creating readonly group

Like we explained any group created will be created with no priviliges at all. Therefore afte creating the readonly group we need to grant them acces to the database, schema and to the select on all the tables in the schema.

In [12]:
cur.execute('''
    CREATE GROUP readonly NOLOGIN;
''')

cur.execute('''
    GRANT CONNECT ON DATABASE crime_db TO readonly;
''')

cur.execute('''
    GRANT USAGE ON SCHEMA crimes TO readonly;
''')

cur.execute('''
    GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;
''')

## Creating readwrite group

Just like the readonly group we will create a new one with the difference that can also insert, update and delete values on all tables but cannot drop them.

In [13]:
cur.execute('''
    CREATE GROUP readwrite NOLOGIN;
''')

cur.execute('''
    GRANT CONNECT ON DATABASE crime_db TO readwrite;
''')

cur.execute('''
    GRANT USAGE ON SCHEMA crimes TO readwrite;
''')

cur.execute('''
    GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;
''')

## Creating one user for each group

We will create two users:

* Data Analyst: with password 'secret1' in the readonly group.


* Data Scientist: with password 'secret2' in the readwrite group

In [14]:
cur.execute('''
    CREATE USER data_analyst WITH PASSWORD 'secret1';
''')

cur.execute('''
    GRANT readonly TO data_analyst;
''')

In [15]:
cur.execute('''
    CREATE USER data_scientist WITH PASSWORD 'secret2';
''')

cur.execute('''
    GRANT readwrite TO data_scientist;
''')

## Database setup test

Let's test the database setup by querying pg_roles table and information_schema.table_privileges.

In the pg_roles table we will check database related privileges and for that we will look at the following columns:

* rolname: The name of the user / group that the privilege refers to.
* rolsuper: Whether this user / group is a super user. It should be set to False on every user / group that we have created.
* rolcreaterole: Whether user / group can create users, groups or roles. It should be False on every user / group that we have created.
* rolcreatedb: Whether user / group can create databases. It should be False on every user / group that we have created.
* rolcanlogin: Whether user / group can login. It should be True on the users and False on the groups that we have created.

In the information_schema.table_privileges we will check privileges related to SQL queries on tables. We will list the privileges of each group that we have created.

In [30]:
conn = psycopg2.connect(dbname = 'crime_db', user = 'postgres', password = '123456789')
cur = conn.cursor()

pg_roles_df = pd.read_sql("""
    SELECT 
        rolname,
        rolsuper,
        rolcreaterole,
        rolcreatedb,
        rolcanlogin
    FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist')
""", conn)

pg_roles_df

,rolname,rolsuper,rolcreaterole,rolcreatedb,rolcanlogin
0,readonly,False,False,False,False
1,readwrite,False,False,False,False
2,data_analyst,False,False,False,True
3,data_scientist,False,False,False,True


In [33]:
conn = psycopg2.connect(dbname = 'crime_db', user = 'postgres', password = '123456789')
cur = conn.cursor()

info_schema_privileges = pd.read_sql("""
    SELECT 
        grantee,
        privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite')
""", conn)

info_schema_privileges

,grantee,privilege_type
0,readonly,SELECT
1,readwrite,INSERT
2,readwrite,SELECT
3,readwrite,UPDATE
4,readwrite,DELETE


As shown by the table above, readonly can only use the SELECT statement while the readwrite can SELECT, INSERT, UPDATE and DELETE.